Project Proposal: https://docs.google.com/document/d/1rRzJV0hOaNMsBIjji8v1Cv8C3QUuZ6lUvYokZ5dif-A/edit


In [1]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
#import sys

credentials = []
f = open('credentials.txt', 'r')
for line in f:
    #add line to credentials list (remove trailing whitespace characters)
    credentials.append(line.rstrip())

# create Spotify object with credentials
client_credentials_manager = SpotifyClientCredentials(credentials[0], credentials[1])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
df5 = pd.read_csv("/Users/gregwelliver/Desktop/springboard_files/Capstone Two - Music/Data/Merged_Data3.csv", index_col=[0])

In [3]:
df5.isna().sum()

track_id                 0
track_name               0
track_genre              0
artist_name              0
track_popularity         0
popular_year        253233
Top100              253233
key                      0
acousticness             0
danceability             0
duration_ms              0
energy                   0
instrumentalness         0
liveness                 0
loudness                 0
modality                 0
speechiness              0
tempo                    0
time_signature           0
valence                  0
second_artist       230971
artist_country       71786
dtype: int64

## EDA

In [4]:
### QUESTION: when we have a categorical independent variable, is there much EDA that we can do with it?
### QUESTION: what do with variables we know are categorical, but python is reading as continuous (e.g. key, modality)

In [5]:
#df4.popular_year = int(df4.popular_year)

In [6]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255096 entries, 0 to 355118
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          255096 non-null  object 
 1   track_name        255096 non-null  object 
 2   track_genre       255096 non-null  object 
 3   artist_name       255096 non-null  object 
 4   track_popularity  255096 non-null  float64
 5   popular_year      1863 non-null    float64
 6   Top100            1863 non-null    object 
 7   key               255096 non-null  float64
 8   acousticness      255096 non-null  float64
 9   danceability      255096 non-null  float64
 10  duration_ms       255096 non-null  float64
 11  energy            255096 non-null  float64
 12  instrumentalness  255096 non-null  float64
 13  liveness          255096 non-null  float64
 14  loudness          255096 non-null  float64
 15  modality          255096 non-null  float64
 16  speechiness       25

In [7]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255096 entries, 0 to 355118
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          255096 non-null  object 
 1   track_name        255096 non-null  object 
 2   track_genre       255096 non-null  object 
 3   artist_name       255096 non-null  object 
 4   track_popularity  255096 non-null  float64
 5   popular_year      1863 non-null    float64
 6   Top100            1863 non-null    object 
 7   key               255096 non-null  float64
 8   acousticness      255096 non-null  float64
 9   danceability      255096 non-null  float64
 10  duration_ms       255096 non-null  float64
 11  energy            255096 non-null  float64
 12  instrumentalness  255096 non-null  float64
 13  liveness          255096 non-null  float64
 14  loudness          255096 non-null  float64
 15  modality          255096 non-null  float64
 16  speechiness       25

In [8]:
df5['popular_year'] = df5['popular_year'].fillna(0)
df5['popular_year'] = df5['popular_year'].astype(int)

#### Selection of independent variable:

The goal of our model is to determine if a song will place on the Top 100 in a given year.  This is represented byt he Top100 variable. Let's explore the indepdent variable further.

In [9]:
# subset of tracks that made top 100
TopSongs = df4[df4['Top100'] == "Y"]

NameError: name 'df4' is not defined

In [ ]:
TopSongs.info()

In [ ]:
print(TopSongs[['track_name', 'second_artist']])

In [ ]:
Artists_Grouped = TopSongs.groupby('artist_name')
Artists_Grouped.first()

In [ ]:
Artists_Grouped = TopSongs.groupby('artist_name').count().track_id.sort_values(ascending=False)
#Artists_Grouped = pd.DataFrame(TopSongs.groupby('artist_name').count().track_id.sort_values(ascending=False))
Artists_Grouped

#### Let's look at more detail on which artist had the most Top 100 hits - Drake.

In [ ]:
DrakeSongs = TopSongs[TopSongs['artist_name'] == "Drake"]
#DrakeSongs = DrakeSongs.groupby('popular_year').count().artist_name.sort_values(ascending=False)
#print(DrakeSongs)
DrakeSongs


In [ ]:
#x = DrakeSongs['popular_year']
#y = DrakeSongs[]

In [ ]:
plt.hist(DrakeSongs.popular_year, bins=20)
plt.xlabel('year')
plt.ylabel('count');

2018 was an amazing year for Drake!

In [ ]:
sns.countplot(x='popular_year', data=DrakeSongs)
plt.xticks(rotation=90);

In [ ]:
#artist_count = TopSongs.groupby('artist_name').count().track_id.sort_values(ascending=False)
#plt.hist(artist_count)

In [ ]:
Genres_Grouped = TopSongs.groupby('track_genre').count().track_id.sort_values(ascending=False)
Genres_Grouped

sns.set_theme(style="white")

sns.relplot(x=TopSongs["popular_year"], y=TopSongs["track_genre"], hue="origin", size="weight",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=6, data=TopSongs)

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,7))
pop_stats = TopSongs.groupby('track_genre').count().track_id.sort_values(ascending=False)
sns.barplot(x=pop_stats.index, y=pop_stats)
;
# QUESTION: how plot this such that "count" shows up on the y-axis

In [ ]:
sns.countplot(x=Genres_Grouped.index, data=Genres_Grouped)
plt.xticks(rotation=90);

In [ ]:
TopSongs.head()

#### come back to this one
with sns.axes_style('white'):
    g = sns.factorplot("popular_year", data=TopSongs, aspect=4.0, kind='count',
                       hue='track_genre', order=range(2000, 2021))
    g.set_ylabels('Count')

In [ ]:
sns.displot(TopSongs, x="popular_year", hue="track_genre");

In [ ]:
sns.displot(TopSongs, x="popular_year", hue="track_genre", multiple="stack", bins=22);

In [ ]:
TopSongs.popular_year.value_counts()

In [ ]:
TopSongs.groupby('artist_name').count().track_id.sort_values(ascending=False)

In [ ]:
TopSongs.groupby(['artist_name', 'popular_year']).count().track_id.sort_values(ascending=False)

In [ ]:
TopSongs.groupby(['artist_name', 'popular_year']).count().track_id.sort_values(ascending=False)

In [ ]:
df4.corr()

### Observations on correlation matrix: 
there does not appear to be much correlation between the availble variables.  The only variables with a coefficient higher than +_0.5 are:
 - Energy/Acousticness
 - Danceability/Valence
 - Loudness/Accousticness
 - Energy/Loudness
 - Acousticness/Loudness
 - Loudness/Instrumentalness
 
 It's worth noting that Loudness is included in four of these pairs (could be a candidate for removal)

In [ ]:
df4.describe()

In [ ]:
#sns.pairplot(df4)

In [ ]:
#seaborn.histplot(data, x, y, hue, stat, bins, binwidth, discrete, kde, log_scale)

sns.histplot(df4.key)


sns.histplot(data=df4.key, x="sepal_length")
plt.show()

In [ ]:
df4.hist(figsize=(15,10))
plt.subplots_adjust(hspace=0.5)

In [ ]:
#boxplot_data = pd.DataFrame(df4.drop(columns='duration_ms',inplace=True))
#df4.liveness.boxplot(figsize=(15,8))
#plt.show()
#boxplot_data.head()

def
for column in df4:
    quality_normal = 

column_list = list(df4.columns)

[stats.normaltest(df4[(column)]) for column in column_list]

plt.hist(df4.key)
plt.xlabel('key')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.key)
print(quality_normal)

In [ ]:
stats.normaltest(df4['acousticness'])

In [ ]:
plt.hist(df4.acousticness)
plt.xlabel('acousticness')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.acousticness)
print(quality_normal)

In [ ]:
plt.hist(df4.danceability)
plt.xlabel('danceability')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.danceability)
print(quality_normal)

In [ ]:
plt.hist(df4.duration_ms)
plt.xlabel('duration_ms')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.duration_ms)
print(quality_normal)

In [ ]:
plt.hist(df4.energy)
plt.xlabel('energy')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.energy)
print(quality_normal)

In [ ]:
plt.hist(df4.instrumentalness)
plt.xlabel('instrumentalness')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.instrumentalness)
print(quality_normal)

In [ ]:
plt.hist(df4.liveness)
plt.xlabel('liveness')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.liveness)
print(quality_normal)

In [ ]:
plt.hist(df4.loudness)
plt.xlabel('loudness')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.loudness)
print(quality_normal)

In [ ]:
plt.hist(df4.modality)
plt.xlabel('modality')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.modality)
print(quality_normal)

In [ ]:
plt.hist(df4.speechiness)
plt.xlabel('speechiness')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.speechiness)
print(quality_normal)

In [ ]:
plt.hist(df4.tempo)
plt.xlabel('tempo')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.tempo)
print(quality_normal)

In [ ]:
plt.hist(df4.time_signature)
plt.xlabel('time_signature')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.time_signature)
print(quality_normal)

In [ ]:
plt.hist(df4.valence)
plt.xlabel('valence')
plt.ylabel('count');

quality_normal = stats.normaltest(df4.valence)
print(quality_normal)

For feature engineering, can derive:
- artist age from kaggle file that I found
- collaboration (based on second artist, or based on "featuring")
- if they previously appeared on a top 100 list

#### write to CSV
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Capstone Two - Music/Data/Merged_Data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df3.to_csv(filepath)

df4 = df4.rename(columns = {'mode':'modality'})